In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
from pathlib import Path

import py3Dmol as p3d

AU2KCAL = 627.5094740630558

In [2]:
def getEnergy(j: Path) -> float:
    """Return total energy from JSON file."""
    with j.open() as f:
        output = json.loads(f.read())
    return output['output']['properties']['scf_energy']['E_tot']


def readXYZ(file: Path) -> str:
    """Return XYZ file contents as string."""
    with file.open() as f:
        return f.read()

## Collect data and compute reaction energies

In [3]:
dir_zora = Path.cwd().joinpath('calcs_zora')
dir_nrel = Path.cwd().joinpath('calcs_nrel')

# Collect data
table_nrel = []
table_zora = []
for f in dir_zora.glob('*.json'):
    id_ = f.stem
    mol, frag, preclabel = id_.split('_')
    e = getEnergy(f)
    table_zora.append((id_, mol, frag, preclabel, e))

for f in dir_nrel.glob('*.json'):
    id_ = f.stem
    mol, frag, preclabel = id_.split('_')
    e = getEnergy(f)
    table_nrel.append((id_, mol, frag, preclabel, e))
    
# Make DataFrames
cols = ['ID', 'Reaction', 'Fragment', 'PrecLabel', 'TotalEnergy']
zora = pd.DataFrame(table_zora, columns=cols).sort_values(by=['Reaction', 'Fragment']).reset_index(drop=True)
nrel = pd.DataFrame(table_nrel, columns=cols).sort_values(by=['Reaction', 'Fragment']).reset_index(drop=True)

# Pivot DataFrames
zora = pd.pivot_table(zora, values='TotalEnergy', columns='Fragment', index='Reaction')
nrel = pd.pivot_table(nrel, values='TotalEnergy', columns='Fragment', index='Reaction')

# Compute reaction energies
zora['ReactionEnergy'] = (zora.Complex - zora.Fragment1 - zora.Fragment2) * AU2KCAL
nrel['ReactionEnergy'] = (nrel.Complex - nrel.Fragment1 - nrel.Fragment2) * AU2KCAL

diff = nrel - zora

## Comparison of reaction energies

In [4]:
# ZORA energies
print(zora.to_latex())

\begin{tabular}{lrrrr}
\toprule
Fragment &      Complex &    Fragment1 &   Fragment2 &  ReactionEnergy \\
Reaction   &              &              &             &                 \\
\midrule
Cr-Alkene1 & -1699.852817 & -1621.254527 &  -78.558078 &      -25.233499 \\
Cr-MeOH    & -1737.036365 & -1621.254527 & -115.752929 &      -18.140480 \\
Cr-THF     & -1853.729814 & -1621.254527 & -232.444315 &      -19.435322 \\
Fe-MeOH    & -3063.948578 & -2948.172431 & -115.752929 &      -14.569730 \\
Ni-Alkene1 & -1946.211785 & -1867.625856 &  -78.558078 &      -17.476684 \\
Ni-MeOH    & -1983.389916 & -1867.625856 & -115.752929 &       -6.984358 \\
Ni-NHC1    & -2172.455631 & -1867.625856 & -304.769788 &      -37.642531 \\
Ni-THF     & -2100.082718 & -1867.625856 & -232.444315 &       -7.873471 \\
\bottomrule
\end{tabular}



In [5]:
# Non-relativistic energies
print(nrel.to_latex())

\begin{tabular}{lrrrr}
\toprule
Fragment &      Complex &    Fragment1 &   Fragment2 &  ReactionEnergy \\
Reaction   &              &              &             &                 \\
\midrule
Cr-Alkene1 & -1689.278481 & -1610.730249 &  -78.508417 &      -24.984060 \\
Cr-MeOH    & -1726.401682 & -1610.730249 & -115.642495 &      -18.159158 \\
Cr-THF     & -1843.020883 & -1610.730249 & -232.259491 &      -19.542192 \\
Fe-MeOH    & -3046.433039 & -2930.767498 & -115.642495 &      -14.461965 \\
Ni-Alkene1 & -1926.579498 & -1848.045060 &  -78.508417 &      -16.327864 \\
Ni-MeOH    & -1963.698947 & -1848.045060 & -115.642495 &       -7.148634 \\
Ni-NHC1    & -2152.652469 & -1848.045060 & -304.549110 &      -36.582752 \\
Ni-THF     & -2080.317452 & -1848.045060 & -232.259491 &       -8.095297 \\
\bottomrule
\end{tabular}



In [6]:
# Energy differences between NREL and ZORA
print(diff.to_latex())

\begin{tabular}{lrrrr}
\toprule
Fragment &    Complex &  Fragment1 &  Fragment2 &  ReactionEnergy \\
Reaction   &            &            &            &                 \\
\midrule
Cr-Alkene1 &  10.574336 &  10.524278 &   0.049661 &        0.249439 \\
Cr-MeOH    &  10.634683 &  10.524278 &   0.110435 &       -0.018679 \\
Cr-THF     &  10.708931 &  10.524278 &   0.184824 &       -0.106870 \\
Fe-MeOH    &  17.515539 &  17.404932 &   0.110435 &        0.107765 \\
Ni-Alkene1 &  19.632287 &  19.580796 &   0.049661 &        1.148820 \\
Ni-MeOH    &  19.690969 &  19.580796 &   0.110435 &       -0.164275 \\
Ni-NHC1    &  19.803163 &  19.580796 &   0.220678 &        1.059779 \\
Ni-THF     &  19.765266 &  19.580796 &   0.184824 &       -0.221826 \\
\bottomrule
\end{tabular}



In [7]:
reactions = zora.index
reactions.values
e_zora = zora.ReactionEnergy
e_nrel = nrel.ReactionEnergy
e_diff = diff.ReactionEnergy

data = pd.DataFrame({"ZORA": e_zora, "NREL": e_nrel, "ZORA - NREL": e_diff}).reset_index()
print(data.to_latex(index=False))

\begin{tabular}{lrrr}
\toprule
  Reaction &       ZORA &       NREL &  ZORA - NREL \\
\midrule
Cr-Alkene1 & -25.233499 & -24.984060 &     0.249439 \\
   Cr-MeOH & -18.140480 & -18.159158 &    -0.018679 \\
    Cr-THF & -19.435322 & -19.542192 &    -0.106870 \\
   Fe-MeOH & -14.569730 & -14.461965 &     0.107765 \\
Ni-Alkene1 & -17.476684 & -16.327864 &     1.148820 \\
   Ni-MeOH &  -6.984358 &  -7.148634 &    -0.164275 \\
   Ni-NHC1 & -37.642531 & -36.582752 &     1.059779 \\
    Ni-THF &  -7.873471 &  -8.095297 &    -0.221826 \\
\bottomrule
\end{tabular}



## Chromium-THF Complex

In [8]:
data = readXYZ(Path.cwd().joinpath('xyz_files/Cr-THF_Complex.xyz'))
p = p3d.view(width=400, height=400)
p.addModel(data, 'xyz')
p.setStyle({'stick': {}})
p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Nickel-NHC Complex

In [9]:
data = readXYZ(Path.cwd().joinpath('xyz_files/Ni-NHC1_Complex.xyz'))
p = p3d.view(width=400, height=400)
p.addModel(data, 'xyz')
p.setStyle({'stick': {}})
p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Fe-Pincer-MeOH Complex

In [10]:
data = readXYZ(Path.cwd().joinpath('xyz_files/Fe-MeOH_Complex.xyz'))
p = p3d.view(width=400, height=400)
p.addModel(data, 'xyz')
p.setStyle({'stick': {}})
p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Generate SI data

In [11]:
zora["Relativity"] = "ZORA"
nrel["Relativity"] = "None"

zora.reset_index(inplace=True)
nrel.reset_index(inplace=True)

zora.columns.name = None
nrel.columns.name = None

df = pd.concat([zora, nrel], axis=0)
df["TransitionMetal"] = list(map(lambda x: x[0], df.Reaction.str.split("-")))
df["Ligand"] = list(map(lambda x: x[1], df.Reaction.str.split("-")))

df.ReactionEnergy = df.Complex - df.Fragment1 - df.Fragment2

df.sort_values(by=["TransitionMetal", "Ligand"], inplace=True)
df.reset_index(drop=True, inplace=True)

mapping = {"Cr": "(CO)5", "Ni": "(CO)3", "Fe": "-Pincer"}

df.rename(columns={"Complex": "EnergyComplex", 
                   "Reaction": "Complex", 
                   "Fragment1": "EnergyFragment1",
                   "Fragment2": "EnergyFragment2"},
         inplace=True)

for idx, row in df.iterrows():
    s = row.TransitionMetal + mapping[row.TransitionMetal] + "-" + row.Ligand
    df.Complex[idx] = s
    
df["MWPrecision"] = 1e-6

<ipython-input-11-63b5dd233d07>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Complex[idx] = s


In [12]:
df.to_csv("data_reaction_energies.csv", index=False)